In [1]:
import pandas as pd
import random
import math
import plotly


In [782]:
def punch_card(raw_data_x, 
               raw_data_y, 
               raw_data_values, 
               raw_data_colors=None, 
               verbose=False,
               validate=True,
               layout=dict(),
               outer_color='rgb(0, 0, 0)',
               outer_width=2,
               size_format='.2f',
               size_marker=dict(min='min', 
                               middle='mean', 
                               max='max'),
               fillcolor='blue',
               number_format='.2f',
               place_markers=True,
               normalize=True,
               max_size=25,
               min_size=5,
               color_bar_len=0.8,
               color_bar_y=0.35,
               marker_symbol='circle',
               equal_size_markers=False,
               colorscale='Jet',
               hoverinfo='x+y+text',
               debug=False,
               other_params=dict()
              ):
    """
    
    raw_data_x: the x-values
    raw_data_y: the y-values
    raw_data_values: the values associated with each x and y value
    raw_data_colors: Either None or an array of colors which are used in addition to raw_data_values which are used for the size, 
        if None and validate=True raw_data_values is used for the colors
    verbose: boolean, prints info messages
    validate: boolean, whether input values should be checked and raw_data_values adjusted to x and y values
    layout: dict, layout parameters which are passed to layout, default: dict()
    
    size_format: the number format in the legend, default=".2f"
    
    normalize: boolean, whether raw_data_values are normalized to min and max (0 to 1), 
        ignored if equal_size_marker is True
    max_size: maximum size of markers (1 in normalized raw_data_values)
    min_size: minimum size of markers (0 in normalized raw_data_values
    equal_size_markers: boolean, markers are all equal size, color is derived from raw_data_values
    colorscale: either a named colorscale or an array with values and colors
    hoverinfo: Plotly parameter
    color_bar_len: Plotly parameter, length of colobar (0 to 1)
    color_bar_y: Plotly parameter, y position of colorbar
    marker_symbol: Plotly parameter, symbol for markers
    debug: bolean, print debug messages
    outer_color: the color circling the dots
    outer_width: the widht of the line circling the dots
    place_markers: whether legend with markers size and colors should appear, default=True
    background: TODO, needed?, the background color your plot  
    """
    
    if validate:
        #take colors from values if needed
        if not raw_data_colors:
            raw_data_colors = raw_data_values.copy()
            if verbose:
                print('Colors are taken from values')
        
        
        #adjust raw_data_x and raw_data_y to the size of raw_data_values
        len_data_values = len(raw_data_values)
        if len(raw_data_y) < len_data_values:
            raw_data_y_new = list()
            for r in range(math.ceil(len_data_values / len(raw_data_y))):
                for i in range(len(raw_data_y)):
                    raw_data_y_new.extend(raw_data_y[i:i+1] * len(raw_data_x))
            raw_data_y = raw_data_y_new[0:len_data_values]
            raw_data_y = (raw_data_y * math.ceil(len_data_values / len(raw_data_y)))[0:len_data_values]        
        if len(raw_data_x) < len_data_values:
            raw_data_x = (raw_data_x * math.ceil(len_data_values / len(raw_data_x)))[0:len_data_values]

    
    size_factor = dict(min=min(raw_data_values),
                       max=max(raw_data_values))
    
    if debug:
        print('raw data values')
        print(raw_data_values)


    cmin = min(raw_data_values)
    cmax = max(raw_data_values)

    size_factor=dict(min=cmin,
                     max=cmax)

    if equal_size_markers:
        relative_size = 1
        sizes = max_size
    elif normalize:
        relative_size = abs(size_factor['max'] - size_factor['min'])
        sizes = [min_size + (raw - size_factor['min']) / relative_size * (max_size - min_size) for raw in raw_data_values]
    else:
        relative_size = 1
        sizes = raw_data_values.copy()
        
    if debug:
        print('sizes')
        print(sizes)
        print('min/max')
        print(cmin, cmax)
    
    all_params = other_params.copy()
    all_params['x'] = raw_data_x
    all_params['y'] = raw_data_y
    all_params['mode'] = 'markers'
    if all_params.get('marker'):
        all_params['marker']['symbol'] = marker_symbol
        all_params['marker']['size'] = sizes
        all_params['marker']['color'] = color=raw_data_values
        all_params['marker']['colorscale'] = colorscale
        all_params['marker']['showscale'] = True
        if all_params['marker'].get('colorbar'):
            all_params['marker']['colorbar'].update(dict(len=color_bar_len, 
                                                         y=color_bar_y)
                                                    )
        else:
            all_params['marker']['colorbar'] = dict(len=color_bar_len, 
                                                    y=color_bar_y)
            
                                               
        if all_params['marker'].get('line'):
            all_params['marker']['line'].update(dict(color=outer_color,
                                                     width=2)
                                               )
        else:
            all_params['marker']['line'] = (dict(color=outer_color,
                                                 width=2)
                                           )
    else:
        all_params['marker'] = dict(symbol=marker_symbol,
                                    size=sizes,
                                    color=raw_data_values,
                                    colorscale=colorscale, 
                                    showscale=True,
                                    colorbar=dict(len=color_bar_len, 
                                                  y=color_bar_y),
                                    line=dict(color=outer_color,
                                              width=2)
                                    )
                               
    
    all_params['text'] = raw_data_values
    all_params['hoverinfo'] = hoverinfo
    all_params['showlegend'] = False
    
                                         
    circles = [plotly.graph_objs.Scatter(all_params)
              ]

    if place_markers:
        size_marker_keys = ('min', 'middle', 'max')
        size_markers = [0, 0, 0]
    else:
        #skip the following blocks
        size_marker_keys = list()
    if validate:
        for k in size_marker_keys:
            if k not in size_marker.keys():
                print('Size marker needs the following keys: min, middle, max')
            if size_marker[k] not in ('min', 'max', 'mean', 'median', 'avg', 'average'):
                print('Size marker can have the following values: min, max, mean, median, avg, average')
               
    if place_markers:
        size_marker_keys = ('min', 'middle', 'max')
    else:
        #skip the following block
        size_marker_keys = list()
        
    for size in size_marker_keys:
        if equal_size_markers:
            size_markers[size_marker_keys.index(size)] = max_size
            #background = size_markers[size_marker_keys.index(size)]
        if size_marker[size] == 'min':
            marker_background = min(raw_data_values)
            marker_name = 'Min:  {0:{number_format}}'.format(marker_background, number_format=number_format)
            if normalize:
                size_markers[0] = min_size
            else:
                size_markers[0] = cmin
            
        elif size_marker[size] == 'max':
            marker_background = max(raw_data_values)
            marker_name = 'Max:  {0:{number_format}}'.format(marker_background, number_format=number_format)
            if normalize:
                size_markers[2] = max_size
            else:
                size_markers[2] = cmax
            
        elif size_marker[size] in ('mean', 'average', 'avg'):
            if 'numpy' in sys.modules:
                #TO DO
                size_markers[1] = sum(raw_data_values) / len(raw_data_values)
            else:
                size_markers[1] = sum(raw_data_values) / len(raw_data_values)
            
            
            marker_name = 'Mean: {0:{number_format}}'.format(size_markers[1], number_format=number_format)
            marker_background = size_markers[1]
            if debug:
                print('##########')
                print((size_markers[1] - cmin))
                print(abs(cmax - cmin))
                print((size_markers[1] - cmin) / abs(cmax - cmin))
                print((max_size - min_size))
            if normalize:          
                size_markers[1] = min_size + ((size_markers[1] - cmin) / abs(cmax - cmin)) * (max_size - min_size)
            
        #TO DO, finish
        elif size_marker[size] == 'median':
            size_markers[1] = _median(raw_data_values)
            marker_background = size_markers[1]
            marker_name = 'Median: {0:{number_format}}'.format(size_markers[1], number_format=number_format)
            size_markers[1] = (size_markers[1] - min_size) / relative_size * max_size
        if equal_size_markers:
            size_markers[size_marker_keys.index(size)] = max_size
            #background = size_markers[size_marker_keys.index(size)]
        
        if debug:
            print(colorscale)
            print((marker_background-cmin)/(cmax-cmin))
            print(cmax-cmin)
            print('marker size')
            print(size_markers[size_marker_keys.index(size)])
        all_params = other_params.copy()
        all_params['mode'] = 'markers'
        
        all_params['x'] = raw_data_x[0:1]
        all_params['y'] = raw_data_y[0:1]
        all_params['name'] = marker_name
        all_params['visible'] = 'legendonly'
        all_params['showlegend'] = True
        all_params['legendgroup'] ='legend_only'
        all_params['hoverinfo'] = 'name'
        marker_params = dict(symbol=marker_symbol, 
                             color = intermediate_color(colorscale, 
                                                        (marker_background-cmin)/(cmax-cmin)
                                                       ),
                             size=size_markers[size_marker_keys.index(size)],
                             line=dict(color=outer_color,
                             width=outer_width))
        if all_params.get('marker'):
            all_params['marker'].update(marker_params)
        else:
            all_params['marker'] = marker_params
                                                 
        circles.append(plotly.graph_objs.Scatter(all_params)
                      )
    
    data = plotly.graph_objs.Data(circles)
    layout = plotly.graph_objs.Layout(**layout)

    return data, layout

def _median(values):
    """taken from https://codereview.stackexchange.com/a/126898
    """
    median = 0
    sortedlist = sorted(values)
    lengthofthelist = len(sortedlist)
    centerofthelist = lengthofthelist / 2
    if len(sortedlist) % 2 == 0:
        temp = 0.0
        medianparties = []
        medianparties = sortedlist[centerofthelist -1 : centerofthelist +1 ]
        for value in medianparties:
            temp += value
            median = temp / 2
        return median
    else:
        return sortedlist[centerofthelist]
    
def intermediate_color(colorscale, new_value):
    """
    """
    
    if isinstance(colorscale, str) and colorscale in plotly.colors.PLOTLY_SCALES:
        colorscale = plotly.colors.PLOTLY_SCALES[colorscale]
    
    if not isinstance(colorscale[0], tuple) and not isinstance(colorscale[0], list):
        for i, color in enumerate(colorscale):
            colorscale[i] = (i/(len(colorscale) - 1), color)
    if isinstance(colorscale[0][1], list):
        for i, color in enumerate(colorscale):
            colorscale[i] = color[1]
    
    min_value = (0,0)
    max_value = (0,0)
    
    #find the closest values in the colorscale
    for color in colorscale:
        if float(color[0]) == float(new_value):
            return color[1]
        if float(color[0]) < new_value:
            min_value = color
        elif float(color[0]) > new_value and max_value[0] == 0:
            max_value = color
            break
    if max_value == (0, 0):
        max_value = colorscale[-1]
    if min_value == (0, 0):
        min_value = colorscale[0]
    
    min_color = plotly.colors.color_parser(min_value[1], plotly.colors.unlabel_rgb)
    max_color = plotly.colors.color_parser(max_value[1], plotly.colors.unlabel_rgb)
    new_color = plotly.colors.find_intermediate_color(min_color, 
                                     max_color,
                                     (new_value - min_value[0])/(max_value[0] - min_value[0]))
    
    return (plotly.colors.label_rgb(new_color))

In [703]:
y_len = 7
y=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
data = dict(x=[i for i in range(1, y_len + 1)], 
            y=y,
            value = [random.randrange(-5,7) for i in range(7 * len(y))]
           )


In [797]:
d, l = punch_card(data['x'], data['y'], data['value'],debug=False)
plotly.plotly.iplot(plotly.graph_objs.Figure(data=d, layout=l))

In [798]:
d, l = punch_card(data['x'], data['y'], data['value'])
plotly.plotly.iplot(plotly.graph_objs.Figure(data=d, layout=l))

In [799]:
d, l = punch_card(data['x'], data['y'], data['value'], normalize=False)
plotly.plotly.iplot(plotly.graph_objs.Figure(data=d, layout=l))

In [800]:
d, l = punch_card(data['x'], data['y'], data['value'], 
                  verbose=True, 
                  max_size=20, 
                  marker_symbol='square', 
                  equal_size_markers=True, 
                  colorscale=colorlover.flipper()['seq']['7']['Greens'])
plotly.plotly.iplot(plotly.graph_objs.Figure(data=d, layout=l))

Colors are taken from values


In [787]:
d, l = punch_card(data['x'], data['y'], data['value'],
                 normalize=False,
                 equal_size_markers=False)
plotly.plotly.iplot(plotly.graph_objs.Figure(data=d, layout=l))


In [802]:
y_len = 7
y=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
data = dict(x=[i for i in range(1, y_len + 1)], 
            y=y,
            value = [random.randrange(0,5) for i in range(7 * len(y))]
           )
greens = [(0, 'rgb(128,128,128)')] 
greens.append((0.2, 'rgb(128,128,128)'))
for i in range(4):
    greens.append((i * 0.2 + 0.2, colorlover.flipper()['seq']['4']['Greens'][i]))
    greens.append((i * 0.2 + 0.4, colorlover.flipper()['seq']['4']['Greens'][i]))

d, l = punch_card(data['x'], data['y'], data['value'], 
                  verbose=True, 
                  max_size=20, 
                  marker_symbol='square', 
                  equal_size_markers=True, 
                  outer_color='rgb(0,0,0)',
                  colorscale=greens,
                  number_format='.0f',
                  layout=dict(width=400,
                              xaxis=dict(showgrid=False), 
                              yaxis=dict(showgrid=False)
                             )
                 )


plotly.plotly.iplot(plotly.graph_objs.Figure(data=d, layout=l))

Colors are taken from values


In [803]:
reds.append((0.2, 'rgb(200,200,200)'))
for i in range(4):
    reds.append((i * 0.2 + 0.2, colorlover.flipper()['seq']['4']['Reds'][i]))
    reds.append((i * 0.2 + 0.4, colorlover.flipper()['seq']['4']['Reds'][i]))

d, l = punch_card(data['x'], data['y'], data['value'], 
                  verbose=True, 
                  max_size=20, 
                  marker_symbol='square', 
                  equal_size_markers=True, 
                  outer_color='rgb(0,0,0)',
                  colorscale=reds,
                  number_format='.0f',
                  layout=dict(width=400,
                              plot_bgcolor='black',
                              paper_bgcolor='black',
                              xaxis=dict(showgrid=False, tickfont=dict(color='white')), 
                              yaxis=dict(showgrid=False, tickfont=dict(color='white')),
                              legend=dict(font=dict(color='white'))
                             ),
                  other_params=dict(textfont=dict(color='white'), marker=dict(colorbar=dict(tickfont=dict(color='white'))))
                 )



plotly.plotly.iplot(plotly.graph_objs.Figure(data=d, layout=l))

Colors are taken from values
